# Optimización de Hiperparámetros con GridSearchCV para KNN

Explicación del Código Existente
El código proporcionado implementa una búsqueda de hiperparámetros para un clasificador KNN (K-Nearest Neighbors) usando GridSearchCV. Vamos a analizarlo y completarlo:


https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee/

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

# 1. Cargar datos (ejemplo con datos iris)
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
y = data.target

# 2. Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Crear pipeline con escalado y modelo KNN
knn_pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('knn', KNeighborsClassifier())
])

# 4. Definir espacio de búsqueda de hiperparámetros
param_grid = {
    'knn__n_neighbors': range(3, 15, 2),
    'knn__weights': ['uniform', 'distance'],
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'knn__leaf_size': [10, 20, 30, 40],
    'knn__p': [1, 2]  # 1: distancia Manhattan, 2: distancia Euclidiana
}

# 5. Configurar GridSearchCV
gs_knn = GridSearchCV(
    estimator=knn_pipe,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,  # Usar todos los núcleos del CPU
    verbose=1
)

# 6. Ajustar el modelo
gs_knn.fit(X_train, y_train)

# 7. Resultados
print("\nMejores parámetros encontrados:")
print(gs_knn.best_params_)

print("\nMejor score de validación cruzada:")
print(f"{gs_knn.best_score_:.4f}")

# 8. Evaluar en conjunto de entrenamiento y prueba
train_score = gs_knn.score(X_train, y_train)
test_score = gs_knn.score(X_test, y_test)
print(f"\nTraining accuracy: {train_score:.4f}")
print(f"Test accuracy: {test_score:.4f}")

# 9. Reporte de clasificación detallado
y_pred = gs_knn.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 10. Matriz de confusión
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# 11. DataFrame con resultados de la búsqueda
results_df = pd.DataFrame(gs_knn.cv_results_)
print("\nTop 5 combinaciones de hiperparámetros:")
print(results_df.sort_values('rank_test_score').head(5)[[
    'param_knn__n_neighbors',
    'param_knn__weights',
    'param_knn__algorithm',
    'mean_test_score',
    'rank_test_score'
]])

Fitting 5 folds for each of 384 candidates, totalling 1920 fits

Mejores parámetros encontrados:
{'knn__algorithm': 'auto', 'knn__leaf_size': 10, 'knn__n_neighbors': 7, 'knn__p': 1, 'knn__weights': 'uniform'}

Mejor score de validación cruzada:
0.9524

Training accuracy: 0.9333
Test accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45


Confusion Matrix:
[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]

Top 5 combinaciones de hiperparámetros:
    param_knn__n_neighbors param_knn__weights param_knn__algorithm  \
32                       7            uniform                 auto   
34                       7            uniform            

Explicación de Conceptos Clave
1. Hiperparámetros en KNN
Los hiperparámetros son configuraciones del modelo que no se aprenden de los datos:

n_neighbors: Número de vecinos a considerar (K)

weights: Cómo ponderar los vecinos ('uniform' = igual peso, 'distance' = peso por distancia inversa)

algorithm: Algoritmo para computar los vecinos más cercanos

leaf_size: Tamaño de hoja para estructuras de árbol (afecta velocidad)

p: Parámetro de distancia (1=Manhattan, 2=Euclidiana)

2. GridSearchCV
Técnica de búsqueda exhaustiva que:

Prueba todas las combinaciones posibles de hiperparámetros

Usa validación cruzada para evaluar cada combinación

Selecciona la combinación con mejor rendimiento

3. Pipeline
Secuencia de transformaciones con un estimador final:

MinMaxScaler: Normaliza características al rango [0,1]

KNeighborsClassifier: Modelo KNN

4. Ventajas de este enfoque
Optimización sistemática: Encuentra la mejor combinación de parámetros

Prevención de overfitting: Usa validación cruzada

Reproducibilidad: Proceso automatizado y documentado

Mejor rendimiento: Modelo ajustado específicamente para los datos

5. Interpretación de Resultados
best_params_: La mejor combinación encontrada

best_score_: El mejor score de validación cruzada

cv_results_: Resultados detallados de todas las combinaciones

score(): Rendimiento en datos de entrenamiento/prueba

Recomendaciones Prácticas
Rangos de búsqueda:

Comienza con rangos amplios y luego afina

Para n_neighbors, prueba valores impares (evita empates)

Métricas alternativas:

Considera usar 'f1_weighted' para problemas multiclase desbalanceados

Optimización de tiempo:

Usa n_jobs=-1 para paralelizar la búsqueda

Reduce cv si los datos son grandes

Validación final:

Siempre evalúa en un conjunto de test independiente

Compara con el score de validación cruzada

### 🔍 **Diferencia entre Parámetros e Hiperparámetros en Machine Learning**

| **Característica**       | **Parámetros**                                  | **Hiperparámetros**                          |
|--------------------------|-----------------------------------------------|---------------------------------------------|
| **Definición**           | Variables **aprendidas** por el modelo durante el entrenamiento. | Variables **configuradas antes** del entrenamiento. |
| **Quién los define**     | El modelo (ajuste automático con los datos).   | El científico de datos (elección manual o mediante búsqueda). |
| **Ejemplos**            | - Pesos en una red neuronal.<br>- Coeficientes en regresión lineal. | - Número de vecinos en KNN.<br>- Tasa de aprendizaje en redes neuronales. |
| **Objetivo**            | Minimizar la función de pérdida (ej: error).   | Optimizar el rendimiento general del modelo. |
| **Modificación**        | Se actualizan en cada iteración (backpropagation, SGD). | Se prueban combinaciones (GridSearch, RandomSearch). |
| **Influencia**           | Determina **cómo** el modelo hace predicciones. | Determina **qué tan bien** el modelo puede aprender. |

---

### 📌 **Ejemplo Práctico**  
**Modelo: Red Neuronal**  
- **Parámetros**: Pesos de las conexiones entre neuronas (ajustados durante el entrenamiento).  
- **Hiperparámetros**:  
  - Número de capas ocultas.  
  - Función de activación (ReLU, sigmoide).  
  - Tasa de aprendizaje (`learning_rate`).  

---

### 🎯 **¿Por qué importa la diferencia?**  
1. **Parámetros**:  
   - Si el modelo no converge, revisa el algoritmo de optimización o los datos.  
2. **Hiperparámetros**:  
   - Si el modelo no generaliza bien, ajusta hiperparámetros (ej: regularización). 